In [1]:
import os
import glob
from osgeo import gdal
import numpy as np

# 设置GDAL不要压缩输出文件
gdal.SetConfigOption('COMPRESS_OVERVIEW', 'NONE')
gdal.UseExceptions()  # 启用GDAL异常

def process_file(input_file, output_file, target_srs='EPSG:4544', target_res=(0.1, 0.1)):
    try:
        # 如果输出文件已存在，跳过处理
        if os.path.exists(output_file):
            print(f"输出文件已存在，跳过处理: {output_file}")
            return output_file

        # 打开输入文件
        ds = gdal.Open(input_file)
        if ds is None:
            print(f"无法打开文件: {input_file}")
            return None

        # 设置转换选项
        options = gdal.WarpOptions(
            dstSRS=target_srs,
            xRes=target_res[0], yRes=target_res[1],
            resampleAlg=gdal.GRA_Bilinear,
            format='GTiff',
            creationOptions=['COMPRESS=LZW', 'TILED=YES', 'BLOCKXSIZE=256', 'BLOCKYSIZE=256', 'BIGTIFF=YES'],
            warpOptions=['CUTLINE_ALL_TOUCHED=TRUE'],
            multithread=True,
            warpMemoryLimit=4096  # 限制内存使用为4GB
        )

        # 执行转换
        print(f"正在处理文件: {input_file}")
        gdal.Warp(output_file, ds, options=options)
        
        # 关闭数据集
        ds = None
        print(f"成功处理文件: {input_file}")
        return output_file

    except Exception as e:
        print(f"处理文件 {input_file} 时出错: {str(e)}")
        return None

def process_and_merge_images(input_folder, output_file):
    # 获取输入文件夹中所有的tif和ecw文件
    input_files = glob.glob(os.path.join(input_folder, '**', '*.tif'), recursive=True) + \
                  glob.glob(os.path.join(input_folder, '**', '*.ecw'), recursive=True)

    if not input_files:
        print("没有找到TIF或ECW文件")
        return

    # 创建一个列表来存储处理后的文件
    processed_files = []

    for input_file in input_files:
        output_file_temp = os.path.splitext(input_file)[0] + '_processed.tif'
        processed = process_file(input_file, output_file_temp)
        if processed:
            processed_files.append(processed)

    if not processed_files:
        print("没有成功处理的文件")
        return

    try:
        # 合并所有处理后的文件
        print("开始合并处理后的文件")
        options = gdal.WarpOptions(
            format='GTiff',
            dstNodata=None,
            creationOptions=['COMPRESS=LZW', 'TILED=YES', 'BLOCKXSIZE=256', 'BLOCKYSIZE=256', 'BIGTIFF=YES'],
            multithread=True,
            warpMemoryLimit=4096  # 限制内存使用为4GB
        )
        
        gdal.Warp(output_file, processed_files, options=options)
        print(f"成功合并文件，输出: {output_file}")

    except Exception as e:
        print(f"合并文件时出错: {str(e)}")

    finally:
        # 清理临时文件
        for file in processed_files:
            try:
                os.remove(file)
                print(f"删除临时文件: {file}")
            except Exception as e:
                print(f"删除临时文件 {file} 时出错: {str(e)}")

    print(f"处理完成，输出文件: {output_file}")
    
    # 使用示例
input_folder = r'F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像'
output_file = r'F:\result\merged_ch.tif'
process_and_merge_images(input_folder, output_file)

In [4]:
import tkinter as tk
from tkinter import filedialog, messagebox
import os
import glob
from osgeo import gdal
import numpy as np
import threading

class ImageProcessorGUI:
    def __init__(self, master):
        self.master = master
        master.title("Image Processor")

        # Input folder
        tk.Label(master, text="Input Folder:").grid(row=0, column=0, sticky="e")
        self.input_entry = tk.Entry(master, width=50)
        self.input_entry.grid(row=0, column=1)
        tk.Button(master, text="Browse", command=self.browse_input).grid(row=0, column=2)

        # Output file
        tk.Label(master, text="Output File:").grid(row=1, column=0, sticky="e")
        self.output_entry = tk.Entry(master, width=50)
        self.output_entry.grid(row=1, column=1)
        tk.Button(master, text="Browse", command=self.browse_output).grid(row=1, column=2)

        # Pixel size
        tk.Label(master, text="Pixel Size (m):").grid(row=2, column=0, sticky="e")
        self.pixel_entry = tk.Entry(master, width=10)
        self.pixel_entry.grid(row=2, column=1, sticky="w")
        self.pixel_entry.insert(0, "0.1")

        # Output coordinate system
        tk.Label(master, text="Output Coordinate System:").grid(row=3, column=0, sticky="e")
        self.srs_entry = tk.Entry(master, width=20)
        self.srs_entry.grid(row=3, column=1, sticky="w")
        self.srs_entry.insert(0, "EPSG:4544")

        # Process button
        self.process_button = tk.Button(master, text="Process Images", command=self.process_images)
        self.process_button.grid(row=4, column=1)

        # Progress bar
        self.progress = tk.StringVar()
        tk.Label(master, textvariable=self.progress).grid(row=5, column=0, columnspan=3)

    def browse_input(self):
        folder = filedialog.askdirectory()
        self.input_entry.delete(0, tk.END)
        self.input_entry.insert(0, folder)

    def browse_output(self):
        file = filedialog.asksaveasfilename(defaultextension=".tif")
        self.output_entry.delete(0, tk.END)
        self.output_entry.insert(0, file)

    def process_images(self):
        input_folder = self.input_entry.get()
        output_file = self.output_entry.get()
        pixel_size = float(self.pixel_entry.get())
        target_srs = self.srs_entry.get()

        if not input_folder or not output_file:
            messagebox.showerror("Error", "Please specify input folder and output file.")
            return

        self.process_button.config(state=tk.DISABLED)
        threading.Thread(target=self.run_processing, args=(input_folder, output_file, pixel_size, target_srs)).start()

    def run_processing(self, input_folder, output_file, pixel_size, target_srs):
        try:
            self.progress.set("Processing images...")
            self.process_and_merge_images(input_folder, output_file, target_srs, (pixel_size, pixel_size))
            self.progress.set("Processing complete!")
            messagebox.showinfo("Success", "Image processing completed successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")
        finally:
            self.process_button.config(state=tk.NORMAL)

    def process_file(self, input_file, output_file, target_srs, target_res):
        try:
            # 如果输出文件已存在，跳过处理
            if os.path.exists(output_file):
                print(f"输出文件已存在，跳过处理: {output_file}")
                return output_file

            # 打开输入文件
            ds = gdal.Open(input_file)
            if ds is None:
                print(f"无法打开文件: {input_file}")
                return None

            # 设置转换选项
            options = gdal.WarpOptions(
                dstSRS=target_srs,
                xRes=target_res[0], yRes=target_res[1],
                resampleAlg=gdal.GRA_Bilinear,
                format='GTiff',
                creationOptions=['COMPRESS=LZW', 'TILED=YES', 'BLOCKXSIZE=256', 'BLOCKYSIZE=256', 'BIGTIFF=YES'],
                warpOptions=['CUTLINE_ALL_TOUCHED=TRUE'],
                multithread=True,
                warpMemoryLimit=4096  # 限制内存使用为4GB
            )

            # 执行转换
            print(f"正在处理文件: {input_file}")
            gdal.Warp(output_file, ds, options=options)
            
            # 关闭数据集
            ds = None
            print(f"成功处理文件: {input_file}")
            return output_file

        except Exception as e:
            print(f"处理文件 {input_file} 时出错: {str(e)}")
            return None

    def process_and_merge_images(self, input_folder, output_file, target_srs, target_res):
        # 获取输入文件夹中所有的tif和ecw文件
        input_files = glob.glob(os.path.join(input_folder, '**', '*.tif'), recursive=True) + \
                    glob.glob(os.path.join(input_folder, '**', '*.ecw'), recursive=True)

        if not input_files:
            print("没有找到TIF或ECW文件")
            return

        # 创建一个列表来存储处理后的文件
        processed_files = []

        for input_file in input_files:
            output_file_temp = os.path.splitext(input_file)[0] + '_processed.tif'
            processed = process_file(input_file, output_file_temp)
            if processed:
                processed_files.append(processed)

        if not processed_files:
            print("没有成功处理的文件")
            return

        try:
            # 合并所有处理后的文件
            print("开始合并处理后的文件")
            options = gdal.WarpOptions(
                format='GTiff',
                dstNodata=None,
                creationOptions=['COMPRESS=LZW', 'TILED=YES', 'BLOCKXSIZE=256', 'BLOCKYSIZE=256', 'BIGTIFF=YES'],
                multithread=True,
                warpMemoryLimit=4096  # 限制内存使用为4GB
            )
            
            gdal.Warp(output_file, processed_files, options=options)
            print(f"成功合并文件，输出: {output_file}")

        except Exception as e:
            print(f"合并文件时出错: {str(e)}")

        finally:
            # 清理临时文件
            for file in processed_files:
                try:
                    os.remove(file)
                    print(f"删除临时文件: {file}")
                except Exception as e:
                    print(f"删除临时文件 {file} 时出错: {str(e)}")

        print(f"处理完成，输出文件: {output_file}")

root = tk.Tk()
app = ImageProcessorGUI(root)
root.mainloop()

正在处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海7.tif
成功处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海7.tif
正在处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇1.tif
成功处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇1.tif
正在处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇2.tif
成功处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇2.tif
正在处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇3.tif
成功处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇3.tif
正在处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇4.tif
成功处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇4.tif
正在处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇5.tif
成功处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇5.tif
正在处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇6.tif
成功处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇6.tif
正在处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇8.tif
成功处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇8.tif
正在处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇9.tif
成功处理文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇9.tif
开始合并处理后的文件
成功合并文件，输出: F:/result/mergedend.tif
删除临时文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海7_processed.tif
删除临时文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇1_processed.tif
删除临时文件: F:/赫章威宁航飞影像返回/威宁县/1/草海镇影像\草海镇2_processed.tif
删除临时文件: F:/赫章威宁航飞影像返回/威宁

In [2]:
# 使用示例
input_folder = r'F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像'
output_file = r'F:\result\merged_ch.tif'
process_and_merge_images(input_folder, output_file)

正在处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海7.tif
成功处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海7.tif
正在处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇1.tif
成功处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇1.tif
正在处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇2.tif
成功处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇2.tif
正在处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇3.tif
成功处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇3.tif
正在处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇4.tif
成功处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇4.tif
正在处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇5.tif
成功处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇5.tif
正在处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇6.tif
成功处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇6.tif
正在处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇8.tif
成功处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇8.tif
正在处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇9.tif
成功处理文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇9.tif
开始合并处理后的文件
成功合并文件，输出: F:\result\merged_ch.tif
删除临时文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海7_processed.tif
删除临时文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇1_processed.tif
删除临时文件: F:\赫章威宁航飞影像返回\威宁县\1\草海镇影像\草海镇2_processed.tif
删除临时文件: F:\赫章威宁航飞影像返回\威宁

In [ ]:
import arcpy

def convert_ecw_to_tif(input_file, output_file):
    arcpy.CopyRaster_management(input_file, output_file, pixel_type="8_BIT_UNSIGNED")

In [ ]:
ecw_path = r'C:\Users\Runker\Desktop\TIF\么站镇\么站镇1.ecw'
output_path = r'C:\Users\Runker\Desktop\TIF\么站镇\么站镇1.tif'

In [ ]:
convert_ecw_to_tif(ecw_path,output_path)

In [ ]:
import arcpy
import os

def convert_ecw_to_tif(input_file, output_file, target_crs='4544', cell_size=0.1):
    try:
        # 检查输入文件是否存在
        if not os.path.exists(input_file):
            raise FileNotFoundError(f"输入文件不存在: {input_file}")
        
        # 检查输出路径是否可写
        output_dir = os.path.dirname(output_file)
        if not os.access(output_dir, os.W_OK):
            raise PermissionError(f"没有写入权限: {output_dir}")
        
        # 设置输出坐标系
        output_sr = arcpy.SpatialReference(int(target_crs))
        
        # 设置环境变量
        arcpy.env.outputCoordinateSystem = output_sr
        arcpy.env.cellSize = cell_size
        
        print(f"开始转换: {input_file} -> {output_file}")
        print(f"目标坐标系: {target_crs}, 目标像元大小: {cell_size}")
        
        # 使用 CopyRaster 转换 ECW 到 TIF
        arcpy.CopyRaster_management(
            in_raster=input_file,
            out_rasterdataset=output_file,
            pixel_type="8_BIT_UNSIGNED",
            nodata_value="255",
            format="TIFF",
            transform="Transform"
        )
        
        print(f"转换完成。输出文件: {output_file}")
        
    except arcpy.ExecuteError:
        print(arcpy.GetMessages(2))
    except Exception as e:
        print(f"发生错误: {str(e)}")

# 使用示例
# input_ecw = r"C:\Users\Runker\Desktop\TIF\么站镇\么站镇1.ecw"
# output_tif = r"C:\Users\Runker\Desktop\TIF\么站镇\么站镇1111.tif"
# convert_ecw_to_tif(input_ecw, output_tif, target_crs='4544', cell_size=0.1)

In [ ]:
input_ecw = r"C:\Users\Runker\Desktop\TIF\么站镇\么站镇2.ecw"
output_tif = r"C:\Users\Runker\Desktop\TIF\么站镇\么站镇1111.tif"

In [ ]:
convert_ecw_to_tif(input_ecw, output_tif)